In [19]:
# -*- coding:utf-8 -*-
import numpy as np
import pandas as pd
import tushare as ts
import matplotlib.pyplot as plt
import time
import datetime
import pymssql
from pandas import DataFrame
from sqlalchemy import create_engine

In [5]:
class MSSQL():
    def __init__(self,host,user,pwd,db):
        self.host = host
        self.user = user
        self.pwd = pwd
        self.db = db

    def __GetConnect(self):
        if not self.db:
            raise(NameError,"没有设置数据库信息")
        self.conn = pymssql.connect(host=self.host,user=self.user,password=self.pwd,database=self.db,charset="utf8")
        cur = self.conn.cursor()
        if not cur:
            raise(NameError,"连接数据库失败")
        else:
            return cur

    def ExecQuery(self,sql):
        cur = self.__GetConnect()
        cur.execute(sql)
        resList = cur.fetchall()

        #查询完毕后必须关闭连接
        self.conn.close()
        return resList

    def ExecNonQuery(self,sql):
        cur = self.__GetConnect()
        cur.execute(sql)
        self.conn.commit()
        self.conn.close()

In [10]:
'''ms = MSSQL(host="47.99.34.248:3433",user="mj",pwd="xlk20180709",db="Datepython")
reslist = ms.ExecQuery("select * from dbo.测试")
for i in reslist:
    print(i)
newsql="update dbo.测试 set name='%s' where id=1"%u'测试'
print(newsql)
ms.ExecNonQuery(newsql.encode('utf-8'))'''

'ms = MSSQL(host="47.99.34.248:3433",user="mj",pwd="xlk20180709",db="Datepython")\nreslist = ms.ExecQuery("select * from dbo.测试")\nfor i in reslist:\n    print(i)\nnewsql="update dbo.测试 set name=\'%s\' where id=1"%u\'测试\'\nprint(newsql)\nms.ExecNonQuery(newsql.encode(\'utf-8\'))'

In [16]:
startdate = datetime.date(2005,1,1)
enddate   = datetime.datetime.now().date()

In [18]:
'''t0 = datetime.datetime.now().strftime("%Y-%m-%d")
for i in range((enddate-startdate).days+1):
    if i%10==0:
        day1=startdate+datetime.timedelta(days=i)
        day2=startdate+datetime.timedelta(days=i+10)
        df = ts.get_k_data('300016',ktype='5',start=day1,end=day2)
df.to_csv('300016.csv')
df = pd.read_csv('300016.csv',index_col=['date','open','close','high','low','volume','code'])
print(df)'''

't0 = datetime.datetime.now().strftime("%Y-%m-%d")\nfor i in range((enddate-startdate).days+1):\n    if i%10==0:\n        day1=startdate+datetime.timedelta(days=i)\n        day2=startdate+datetime.timedelta(days=i+10)\n        df = ts.get_k_data(\'300016\',ktype=\'5\',start=day1,end=day2)\ndf.to_csv(\'300016.csv\')\ndf = pd.read_csv(\'300016.csv\',index_col=[\'date\',\'open\',\'close\',\'high\',\'low\',\'volume\',\'code\'])\nprint(df)'

In [147]:
t0 = datetime.datetime.now().strftime("%Y-%m-%d")
df = ts.get_k_data('300016',ktype='5',start='2018-07-01',end=t0)

df = pd.DataFrame(df,columns=['date','open','high','low','close','volume'])

df['close']  = list(map(lambda b: float(b),df['close']))
df['high']   = list(map(lambda c: float(c),df['high']))
df['low']    = list(map(lambda d: float(d),df['low']))
df['open']   = list(map(lambda e: float(e),df['open']))
df['volume'] = list(map(lambda f: float(f),df['volume']))
df['date']   = list(map(lambda x:datetime.datetime.strptime(x,'%Y-%m-%d %H:%M'),df['date']))

replace=[]

for i in df['date']:
    dt0 = datetime.datetime.strftime(i,"%Y-%m-%d")
    dt1 = datetime.datetime.strftime(i,"%H:%M")
    dt2 = datetime.datetime.strptime(dt1,"%H:%M")
    times = ['10:30','11:30','14:00','15:00']
    T = [datetime.datetime.strptime(t,"%H:%M") for t in times]
    for j in T:
        diff = j - dt2
        #print(diff.days)
        #print(diff.seconds)
        if diff.seconds < 3600 and diff.days >= 0:
            dt2 = j 
            #print(dt2)
    i = datetime.datetime.strptime(dt0+' '+datetime.datetime.strftime(dt2,"%H:%M"),'%Y-%m-%d %H:%M')
    #print(i)
    replace.append(i)

df['date'] = replace

df.to_csv('300016.csv',index=None)
df = pd.read_csv('300016.csv')
#print(df['open'][0])
#tlist = pd.to_datetime(df['date'])
#print(tlist)
#df = df.set_index(tlist)
print(df)

                    date   open   high    low  close  volume
0    2018-06-26 15:00:00  11.20  11.28  11.19  11.26  3120.0
1    2018-06-26 15:00:00  11.26  11.29  11.26  11.29  1087.0
2    2018-06-27 10:30:00  11.25  11.34  11.10  11.28  3739.0
3    2018-06-27 10:30:00  11.28  11.31  11.20  11.28  3680.0
4    2018-06-27 10:30:00  11.28  11.34  11.28  11.28  1131.0
5    2018-06-27 10:30:00  11.28  11.38  11.26  11.37  1601.0
6    2018-06-27 10:30:00  11.37  11.43  11.35  11.36  1465.0
7    2018-06-27 10:30:00  11.36  11.36  11.29  11.31  1036.0
8    2018-06-27 10:30:00  11.31  11.33  11.26  11.27  1451.0
9    2018-06-27 10:30:00  11.27  11.31  11.27  11.31   810.0
10   2018-06-27 10:30:00  11.31  11.35  11.30  11.32   536.0
11   2018-06-27 10:30:00  11.32  11.35  11.31  11.32   511.0
12   2018-06-27 10:30:00  11.32  11.36  11.31  11.34   579.0
13   2018-06-27 10:30:00  11.34  11.34  11.31  11.32   672.0
14   2018-06-27 11:30:00  11.32  11.33  11.30  11.30   784.0
15   2018-06-27 11:30:00

In [153]:
def ohlcsum(df):
    df = df.sort_values(by='date',ascending=False)
    #df = df.sort()
    return {'open': df['open'].values[0],
             'high': df['high'].max(),
             'low': df['low'].min(),
             'close': df['close'].values[-1],
             'volume': df['volume'].sum()
            }

In [155]:
'''if df['date'][0] == df['date'][1]:
    print(True)
else:
    print(False)'''
#print(df['open'][0])
#print(df['high'].max())
#print(df['volume'].sum())
#print(type(df['close']))
#print(df['close'].values[-1])
df0 = df.groupby(['date']).agg(ohlcsum)
print(df0)

                      open   high    low  close   volume
date                                                    
2018-06-26 15:00:00  11.20  11.29  11.19  11.29   4207.0
2018-06-27 10:30:00  11.25  11.43  11.10  11.32  17211.0
2018-06-27 11:30:00  11.32  11.33  11.19  11.22   8211.0
2018-06-27 14:00:00  11.22  11.23  11.11  11.14  12988.0
2018-06-27 15:00:00  11.14  11.21  11.11  11.16  12561.0
2018-06-28 10:30:00  11.15  11.28  11.05  11.08  15793.0
2018-06-28 11:30:00  11.08  11.18  11.04  11.13   9166.0
2018-06-28 14:00:00  11.13  11.20  11.10  11.11   6330.0
2018-06-28 15:00:00  11.11  11.15  10.94  10.97  14444.0
2018-06-29 10:30:00  10.97  11.19  10.95  11.17  18150.0
2018-06-29 11:30:00  11.17  11.30  11.14  11.30  16854.0
2018-06-29 14:00:00  11.30  11.42  11.28  11.40  22420.0
2018-06-29 15:00:00  11.40  11.47  11.35  11.45  17979.0
2018-07-02 10:30:00  11.40  11.53  11.35  11.42  22112.0
2018-07-02 11:30:00  11.42  11.45  11.31  11.31  11143.0
2018-07-02 14:00:00  11.31  11.

In [141]:
'''ohlc_dict = {                                                                                                             
'open':'first',                                                                                                    
'high':'max',                                                                                                       
'low':'min',                                                                                                        
'close': 'last',                                                                                                    
'volume': 'sum'
}
df1 = df.resample('60min',how=ohlc_dict,closed='left', label='left')'''

"ohlc_dict = {                                                                                                             \n'open':'first',                                                                                                    \n'high':'max',                                                                                                       \n'low':'min',                                                                                                        \n'close': 'last',                                                                                                    \n'volume': 'sum'\n}\ndf1 = df.resample('60min',how=ohlc_dict,closed='left', label='left')"